In [ ]:
#%pip install azure-cognitiveservices-speech



In [80]:
import os
import pandas as pd
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import create_csv_agent

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials2.env")

def printmd(string):
    display(Markdown(string))

In [81]:
import os
import azure.cognitiveservices.speech as speechsdk
import openai

In [82]:
file_url = "./data/MadeUpBookings.csv"
df = pd.read_csv(file_url).fillna(value = 0)
print("Rows and Columns:",df.shape)
df.head()

Rows and Columns: (30, 9)


,Booking ID,Guest Name,Location,Room Type,Room Number,Check-in Date,Check-out Date,Total Cost,Payment Status
0,1,John Smith,New York City,Deluxe,101,1/01/2022,1/05/2022,"$1,000",Paid
1,2,Jane Doe,Los Angeles,Standard,102,1/02/2022,1/06/2022,$800,Paid
2,3,Michael Johnson,Chicago,Suite,103,1/03/2022,1/07/2022,"$2,000",Paid
3,4,Emily Davis,Houston,Deluxe,104,1/04/2022,1/08/2022,"$1,000",Unpaid
4,5,Sarah Brown,Phoenix,Standard,105,1/05/2022,1/09/2022,$800,Paid


In [83]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

In [84]:
openai.api_type = "azure"
openai.api_base = "https://openaisvcdataaidemos.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [85]:
SPEECH_KEY = os.getenv("AZURE_SPEECH_KEY")
SPEECH_REGION = os.getenv("AZURE_SPEECH_REGION")

In [86]:
speech_config = speechsdk.SpeechConfig(subscription=SPEECH_KEY, region=SPEECH_REGION)


In [87]:
audio_output_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)

In [88]:
# Should be the locale for the speaker's language.
speech_config.speech_recognition_language="en-US"
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

# The language of the voice that responds on behalf of Azure OpenAI.
speech_config.speech_synthesis_voice_name='en-US-JennyMultilingualNeural'
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_output_config)


In [89]:
# when is Emily Davis checking out, what is her total cost and what is her payment status?
speech_recognition_result = speech_recognizer.recognize_once_async().get()
print(speech_recognition_result.text)

When is Emily Davis checking out? What is her total cost and what is her payment status?


In [90]:
CSV_PROMPT_PREFIX = """
First get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**. 
- **ALWAYS**, as part of your "Final Answer", explain how you got to the answer on a section that starts with: "\n\nExplanation:\n". In the explanation, mention the column names that you used to get to the final answer. 
"""

In [91]:
QUESTION = speech_recognition_result.text

In [92]:
llm = AzureChatOpenAI(deployment_name="GPT4", temperature=0, max_tokens=500)

In [93]:
agent_executor = create_pandas_dataframe_agent(llm=llm,df=df,verbose=True)

In [94]:
agent_executor.agent.allowed_tools

['python_repl_ast']

In [95]:
printmd(agent_executor.agent.llm_chain.prompt.template)


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df_head}

Begin!
Question: {input}
{agent_scratchpad}

In [96]:
# We are doing a for loop to retry N times. This is because: 
# 1) GPT-4 is still in preview and the API is being very throttled and 
# 2) Because the LLM not always gives the answer on the exact format the agent needs and hence cannot be parsed

for i in range(5):
    try:
        response = agent_executor.run(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX) 
        break
    except:
        response = "Error too many failed retries"
        continue
        
print(response)



> Entering new AgentExecutor chain...
Thought: I need to get the column names of the dataframe first.
Action: python_repl_ast
Action Input: list(df.columns)
Observation: ['Booking ID', 'Guest Name', 'Location', 'Room Type', 'Room Number', 'Check-in Date', 'Check-out Date', 'Total Cost', 'Payment Status']
Thought:Now that I have the column names, I can filter the dataframe to find the row with Emily Davis and extract the required information.

Thought: I need to filter the dataframe to get the row with Emily Davis.
Action: python_repl_ast
Action Input: df[df['Guest Name'] == 'Emily Davis']
Observation:    Booking ID   Guest Name Location Room Type  Room Number Check-in Date  \
3           4  Emily Davis  Houston    Deluxe          104     1/04/2022   

  Check-out Date Total Cost Payment Status  
3      1/08/2022     $1,000         Unpaid  
Thought:I have found the row with Emily Davis. Now I can extract the check-out date, total cost, and payment status.

Thought: I need to extract t

In [97]:
def remove_words_after_word(sentence, word):
    words = sentence.split()
    new_sentence = []
    for w in words:
        if w == word:
            break
        new_sentence.append(w)
    return ' '.join(new_sentence)

text2 = response.replace('\n', ' ').replace(' .', '.').strip()
new_sentence = remove_words_after_word(text2, "Explanation")
print('Azure OpenAI response:' + new_sentence)

Azure OpenAI response:Emily Davis is checking out on 1/08/2022. Her total cost is $1,000, and her payment status is Unpaid. Explanation: I filtered the dataframe to find the row with Emily Davis and extracted the check-out date, total cost, and payment status using the column names 'Check-out Date', 'Total Cost', and 'Payment Status'.


In [98]:
speech_synthesis_result = speech_synthesizer.speak_text_async(new_sentence).get()
print("Speech synthesized to speaker for text [{}]".format(new_sentence))

Speech synthesized to speaker for text [Emily Davis is checking out on 1/08/2022. Her total cost is $1,000, and her payment status is Unpaid. Explanation: I filtered the dataframe to find the row with Emily Davis and extracted the check-out date, total cost, and payment status using the column names 'Check-out Date', 'Total Cost', and 'Payment Status'.]
